A place to demo some 3D interactive plotting of brain surfaces

In [ ]:
import numpy as np
import ipyvolume.pylab as p3
import nibabel

In [ ]:
# Read in some data

# Data comes from the MNE-python sample dataset
# Surface data
surf = 'lh.pial'
rr, tris = nibabel.freesurfer.read_geometry(surf)
tris = tris.astype(np.uint32)

# Curvature
curv = 'lh.curv'
curv = nibabel.freesurfer.read_morph_data(curv)

# Normalize curvature
curv = (curv > 0).astype(float)
curv = (curv - 0.5) / 3 + 0.5
curv = curv[:, np.newaxis] * [1, 1, 1]

## ipyvolume

In [ ]:
x, y, z = rr.T

In [ ]:
print(x.shape)
print(tris.shape)
print(curv.shape)

In [ ]:
fig = p3.figure(width=500, height=500)
p3.plot_trisurf(x, y, z, triangles=tris, color=curv)
fig

## pythreejs

XXX: Haven't got this working yet because I think I have the wrong version installed :-)

In [ ]:
from pythreejs import (PlainBufferGeometry, Mesh, PerspectiveCamera, Scene,
                       AmbientLight, Renderer, PhongMaterial, DirectionalLight, OrbitControls)
import numpy as np
from IPython.display import display
import ipywidgets

In [ ]:
vertices = rr
faces = tris
vertexcolors = curv
geom = PlainBufferGeometry(vertices=vertices, faces=faces, colors=vertexcolors)
mesh = Mesh(geometry=geom, material=PhongMaterial(shininess=0,vertexColors='VertexColors'))
camera = PerspectiveCamera(position=[500, 500, 500], fov=20,
                           children=[DirectionalLight(color='#ffffff',
                                                      position=[-3, 5, 1],
                                                      intensity=0.5)])
scene = Scene(children=[mesh, AmbientLight(color='#dddddd')])

renderer = Renderer(camera=camera, background='black', background_opacity=1,
                    scene=scene, controls=[OrbitControls(controlling=camera)])


In [ ]:
norm = np.linalg.norm(vertices, axis=1)
norm /= norm.max()
def color(n):
    newcolor=curv.copy()
    newcolor[:,0] = np.abs(np.sin(norm*n))
    newcolor[:,2] = np.abs(np.cos(norm*n))
    geom.colors = newcolor

In [ ]:
from ipywidgets import FloatSlider, VBox
x = FloatSlider(min=0, max=12)
def slide(change):
    color(x.value)
x.observe(slide)
display(VBox([x, renderer]))

In [ ]:
camera.position

In [ ]:
camera.position = [-908.2539226976295, -0.0193857323769, 200]